<a href="https://colab.research.google.com/github/souceira/UofTHacksX-safetyalertapp/blob/main/ttcalertsbutbetter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone --depth=1 https://github.com/twintproject/twint.git
!cd /content/twint && pip3 install . -r requirements.txt
!pip install cohere
!pip install aiohttp==3.7.0
!pip install nest_asyncio
# Import Libraries
import twint
import nest_asyncio
nest_asyncio.apply()
import time
import pandas as pd
import os
import re
timestr = time.strftime("%Y%m%d")


In [ ]:
import cohere
from cohere.classify import Example
co = cohere.Client('HiviLqEmyrSGCMosPNvtAoPCjTjau03Es3ktO3Cx')
response = co.classify(
  model='large',
  inputs=["The TTC has failed to go a single day without delays or other types of issues they report. They are embarassing #ttcproblems #TTC #Toronto", "Line 2 Bloor-Danforth: Minor delays eastbound from Woodbine to Kennedy while we fix a signal problem.", "I think I’d be fine with this upcoming 10 minute delay thing TTC is doing if they said on the bus “this bus will be out of service for 10 mins” rather than waiting 15 for drivers to change but hey just been a rider for 20 years", "The #TTC is a consistent fucking pile of shit. Toronto is a piece of shit and Canada fucking sucks all the time.", "They are building more subways & causing more gridlock!! The TTC SUCKS & sorry, killing people is not an option!!", "A couple of fights, unprovoked attacks, pushing onto the tracks and a sexual assault in Kennedy station subway. In all the years of riding the TTC and I’ve been doing it since university - I don’t ever recall this many issues in such a short time period.", "Toronto Police responded to an assault call at Bloor Yonge TTC station. First, the suspect struck the victim in the head, causing their turban to come off their head, then the suspect made derogatory comments.", "Toronto police said on Wednesday, at around 8:30 a.m., officers recevied a report of an assault at the TTC Bloor-Yonge Subway Station.", "someone just got shanked at the bloor yonge station dude wtf"],
  examples=[Example("Definitely \"shocking\". From reports that I saw after this; a female who was found stabbed last night @ Dundas Squ. it happened at a Univ. campus (stabbed in the abdomen) and another incident, too. Female saying \"they wished they saw TTC staff to talk to\", not necessarily police.", "avoid"), Example("Sad to hear about the woman pushed to the ground at Yonge and King by a random stranger and she has died. Good they have the person in custody.", "avoid"), Example("Thankfully, the man was not pushed onto the TTC tracks at Bloor earlier today. The man that tried to push him was arrested at Rosedale. Good on the TTC staff who recognized him and called it in. Way too much random violence happening.", "avoid"), Example("Victim of stabbing incident located at Dundas subway station", "avoid"), Example("Here we go again another incident today on TTC property at rosedale subway ! But again our mayor says its safe / I wonder if he would say this if one of his family members got hurt in these incidents ! Car Pool its safer if u can", "avoid"), Example("There was another attack on the TTC this morning. This one was hate motivated towards a specific community. I must have missed the incident by five mins or so. Be safe out there.", "avoid"), Example("Man pushed to tracks in argument at Bloor-Yonge Station in latest incident of TTC violence", "avoid"), Example("Also 2 people stabbed (1 dead) on the TTC, and another few slashed in the face by a knife in a random incident. My city is not safe.", "avoid"), Example("Which TTC assault? In the stabbing incident at High Park station last week the police arrested a 52 year old man.", "avoid"), Example("Police say a man who was reportedly stabbing seats onboard a Toronto subway train Friday afternoon hasn’t been located. No injuries have been reported. The incident comes one day after a fatal stabbing onboard a TTC subway train.", "avoid"), Example("ARREST MADE: Demonstration on Sat, Feb 5 Bedford Rd + Bloor St West Man, 34 arrested after throwing feces at another person. He was charged with: 1. Assault with a Weapon", "avoid"), Example("STABBING: Bloor St W + Roxton Rd at 12:22 am - At/near a bar - Reports of man stabbed multiple times - Unknown how serious - Police o/s - Have located victim - Will be taken to hospital - Officers searching area - 1 man now in custody - Weapon recovered", "avoid"), Example("Witnessed a bunch of young teenager girls attack each other with knives near bloor and ossington", "avoid"), Example("A suspect allegedly attacked a man wearing a turban at Yonge-Bloor subway station this morning", "avoid"), Example("Man arrested after 4 injured in hammer attack at Bloor subway station", "avoid"), Example("2 brothers at Yonge & Dundas Sq. in Toronto, Canada, who were giving da\'wah were violently assaulted by 3 men at 3:15 pm", "avoid"), Example("ASSAULT/STABBING - Hurontario St / Dundas St W Adult male stabbed being taken to trauma centre - Suspect w/ knife fled on foot - M/B, in his 30\'s, large build, all black clothing - Large police presence - Awaiting further information", "avoid"), Example("501 Queen: Minor delays eastbound near Queen St East at Jarvis St while we respond to a medical emergency.", "delay"), Example("Line 1 Yonge-University: Minor delays northbound at Rosedale due to a security incident.", "delay"), Example("101 Downsview Park: Minor delays near Sheppard Ave West at Chesswood Dr due to a blocked road.", "delay"), Example("47 Lansdowne: Detour southbound via College St, Dufferin St and Queen St W while we respond to a security incident.", "delay"), Example("501 Queen: No service between Long Branch Loop and Queen St East at Neville Park Blvd due to a security incident. 501 Queen service turning back at Kingston Road Loop.", "delay"), Example("what’s up with northbound subway? Very slow", "delay"), Example("hi TTC, your yonge university subway line southbound is very much delayed and slow. It is making continual frequent stops. This is really bad service. I am continously delayed using your transit system. Very disappointed.", "delay"), Example("Hi Phil, sorry to hear about your experience this evening. Delays regarding this route were posted on the web.", "delay"), Example("The delay we were experiencing on Line 2 at Islington Station is now clear. The delay was cleared by a TTC Staff Member.", "delay"), Example("Line 3 Scarborough: Minor delays between Kennedy and McCowan while we fix a mechanical problem.", "delay"), Example("Hi, sorry about the wait. The subway is running 3 minutes late and should be there in approx. 2 minutes.", "delay"), Example("Line 3 Scarborough: Minor delays at McCowan while we fix a mechanical problem.", "delay"), Example("501 Queen: Minor delays near Queen St East at Kingston Rd due to construction.", "delay"), Example("501 Queen Regular service has resumed between Dufferin St at Springhurst Ave and Queen St West at Dufferin St.", "delay"), Example("I was glad to get out and laugh tonight...I\'m also glad I got home safe ...I am quite frankly on edge after everything that has happened in my area and then hearing of the stabbing, the issues on TTC ...please be vigilant everyone ", "opinion"), Example("god I hate the TTC", "opinion"), Example("Being unreliable, unhelpful, unsafe, not adjusting to any road emergencies will do it for you. Every single person working for TTC management and the planning department needs to be a regular user of the service.", "opinion"), Example("maybe one day toronto will get its shit together and increase police presence to deter assault", "opinion"), Example("ttc sucks its slow everyday and shit happens all the time", "opinion"), Example("never a day without delay on the ttc woohoo", "opinion"), Example("Honest what hell is going on  gta so many scams,   people  getting  assault  in ttc property,", "opinion"), Example("How the ttc pretends to protect people in their facilities? A camera records events AFTER they happened! I have been victim of verbal assault and left before escalated to more. What about the incident on King station? TTC is not safe!", "opinion"), Example("This should not be assault. Anyone trying to push someone onto a live train track should be charged with attempted murder. It’s seems to violence is continuing on the TTC in 2023. To many criminals, addicted and mentally disturbed people just hanging around on TTC property", "opinion"), Example("Toronto police have released security images of a suspect being sought after a sexual assault that took place in the bathroom at a TTC subway station", "opinion"), Example("It’s amazing to me that we’re 15 months into this and the TTC would still rather pay fare inspectors to harass passengers than hire people to hand out masks and ensure riders are wearing them on TTC vehicles.", "opinion"), Example("because of continued failed communication and policies, this government really going to allow annoying rude ass people to harass others for wearing masks. people are already going at each other’s necks daily on the ttc. what a gd mess", "opinion"), Example("So we gotta pay more for shitty transit but the cops (both inside and outside the TTC) get more money to do nothing but harass poor folks, while 10 more outreach workers are hired to try fix a system that is intentionally broken. Got it.", "opinion"), Example("So many women on TikTok have stories of harassment on the TTC and lord knows the TTC will do absolutely nothing but harass you for the $3.25 How many stories have come out about the TTC this summer? Many. How have they responded? By bringing back fare enforcement officers.", "opinion"), Example("ttc drivers/employees are quick to harass customers or kick them off over bus fare or whatever they assume to be an attitude. but when commuters are being harassed by someone on the bus they\'re quiet and it makes them complicit.", "opinion"), Example("TTC \"lacks\" the recourses to safeguard transit users from being burned alive or pushed off the track but has a host of funds to harass and arrest random fare-dodgers", "opinion"), Example("Half of Toronto subway riders say crowding makes them feel unsafe, poll says", "opinion")])
print('The confidence levels of the labels are: {}'.format(response.classifications))

# web scraper
# twitter API

# store in database? or pipe it into cohere


In [ ]:
#configuration
c = twint.Config()   
c.Limit = 1000       
c.Lang = "en"        # Language
c.Store_csv = True
c.Search = "ttc stabbing"  # key words to look for.
c.Output = "twitter_attack.csv"
twint.run.Search(c)

In [ ]:
stand_df = pd.read_csv("twitter_attack.csv")

# Tweetandtime = stand_df.filter(['tweet', 'time'])

# Tweetandtime.head(10)

Tweet = stand_df.filter(['tweet'])

Tweet.head(10)

In [ ]:
import re
import csv

# create a function to clean the tweets
def cleanTxt(text):
    #Remove @mentions and replace with blank
    text = re.sub(r'@[A-Za-z0–9]+', '', text) 
    
    #Remove the '#' symbol, replace with blank
    text = re.sub(r'#', '', text) 
    
    #Remove the 'â€˜' symbol, replace with blank
    text = re.sub(r'[^A-Za-z0-9 "'':,?.-=]+', '', text)
    
    #Removing RT, replace with blank
    text = re.sub(r'RT[\s]+', '', text) 
    
    #Remove the hyperlinks
    text = re.sub(r'https?:\/\/\S+', '', text) 
 
    return text# clean the text

# Tweetandtime['tweet']= Tweetandtime['tweet'].apply(cleanTxt) #Show the clean text
# cleandata = Tweetandtime.drop_duplicates(subset = ['tweet'])

Tweet['tweet']= Tweet['tweet'].apply(cleanTxt) #Show the clean text
cleandata = Tweet.drop_duplicates(subset = ['tweet']) # this isn't really removing duplicates... if you have a better idea put it in

cleandata.to_csv("cleanedupdata.csv")

In [ ]:
# plugging data into co:here model

from cohere.classify import Example

input_data = ["Yet another violent attack today on the TTC but hey lets make you pay more to use it. Honestly why is the CEO making 438,000 year ? Shouldnt safety be", "Woman killed in suspected random attack in downtown Toronto     Rare, isnt it ? Same can be said about setting people on fire, amp; stabbing them to death in TTC. Stop moonlighting as COVID jab sales guy, do your job.", " reported a hatemotivated attack on transit, releasing a photo of suspect. TTCs CEO, Rick Leary, responds that he wants Streets to Homes workers amp;  affirms his budget proposal for transit cops.   Theyre ASSUMING the suspect is homeless. On what basis?  ", "   yet nowhere did 24 mention that this was an antisemitic attack, and they interviewed some random guy who said it doesnt matter who was targeted. Why??", "There are reports that the hatemotivated assault on the  yesterday was an antisemitic attack. A man allegedly struck another on the head knocking his Kippah off while shouting antisemitic comments.   are investigating.    ","Spending more money in enforcing fares instead of just investing more on the ttc. This is a direct attack on the workers and the masses. These people really dont know what theyre starting","Another attack on the TTC They learned that a man had allegedly struck one person on the head, which resulted in their religious head covering being knocked over and falling to the ground. TOpoli onpoli   ","There was another attack on the TTC this morning. This one was hate motivated towards a specific community. I must have missed the incident by five mins or so. Be safe out there.","Fizza Khalid of Scarborough Environmental Association says TTC cuts are a direct attack on shift workers, who were our pandemic heroes...  I already tend to avoid using transit at offpeak hours. There is safety in numbers. Cuts will mean less safety. ","I dont know about this but he was trying to protect a homeless woman, these girls tried to take her alcohol. Also the same girls were assaulting ppl earlier on the TTC. Such a violent attack its really sad. Too bad they didnt catch them earlier","Thief Steals USD 8.2m From Nexus Mutual Founder in a Targeted Attack. TechTrees TTC","Avalanche Blockchain Used To Implement Flash Loan Attack  370K Nabbed Binance ","Police investigate after teen girls allegedly attack people at TTC ...  CTV News Toronto   Toronto Transit TTC GoTransit GoTrain GoBus Transportation","  It looks as though he was trying to leave but the other guy came at him. What are you supposed to do, allow someone to attack you from behind? Maybe you should become a TTC cop.","How do you feel about riding TTC after the recent girl gang attack on riders?","The teenage girls alleged to have murdered a homeless man downtown in a swarming attack are also suspected in a string of random assaults along the TTCs Line 1 the same night, a source confirmed Wednesday.   ","Police investigating alleged assaults by group of teen girls on TTC, wont say if its related to fatal swarming attack  ","Police investigating alleged assaults by group of teen girls on TTC, wont say if its related to fatal swarming attack    ","Inverse Finance exploited again for 1.2M in flashloan oracle attack BinanceTTC TTC TechTrees","Police investigating alleged assaults by group of teen girls on TTC, wont say if its related to fatal swarming attack  ","Police Investigating Alleged Assaults By Group Of Teen Girls On TTC, Wont Say If Its Related To Fatal Swarming Attack   ","Canada police ID Toronto homeless man, 59, allegedly murdered by 8 teen girls in swarming attack   Toronto TOpoli Crime security Police cops assault Investigation Murder Manslaughter homicide stabbing Teengirls Teenagers Teens girls TTC","Teens charged after Toronto swarming attack also suspects in random TTCassaults    Toronto TOpoli Crime security Police cops assault Investigation Murder Manslaughter homicide stabbing Teengirls Teenagers girls TTC Publictransit Transit","Policeinvestigating alleged assaults by group of teen girls on TTC, wont say if its related to fatal swarming attack   ","Teens charged after Toronto swarming attack also suspects in random TTC assaults: police source  ","Boom no wonder misses kitty said to shave my head Gonna need funding.... and for people to stop hat stealing Another biological attack...... this is why people are not allowed to say they are going to sleep with me at the TTC union station. I said it was not OK even by the pol  ","More Clockwork Orange shit. Police investigating alleged assaults by group of teen girls on TTC, wont say if its related to fatal swarming attack     ","The teen girls alleged to have murdered a homeless man downtown Toronto in a swarming attack are also suspected in a string of random assaults along the TTCs Line 1 the same night, a source confirmed Wednesday  ","UPDATED: The teen girls charged after allegedly murdering a Toronto homeless man in a swarming attack are also suspects in a series of random TTC assaults, a Toronto police source tells Global News.  ","Time to stop protecting criminals: start protecting citizens, people deserve to see who this scum is...  Police investigating alleged assaults by group of teen girls on TTC, wont say if its related to fatal swarming attack at St Patrick Osgoode St. Andrew Union  ","Police investigating alleged assaults by group of teen girls on TTC, wont say if is related to fatal swarming attack   via 24","Completely senseless and brutal. Nothing will reform these young girls. Teens part of Toronto swarming attack suspects in TTC assaults  The Star  ","The teen girls alleged to have murdered a homeless man downtown in a swarming attack are also suspected in a string of random assaults along the TTCs Line 1 the same night, a source confirmed Wednesday.  ","The teen girls alleged murderers of a homeless Chinese man are also suspected in string of random attack along the TTC. Parents should have eyes on their kids.","Ransomware attack exposes California transit giants sensitive data  Cybersecurity Dive:   TTC GoTransit GoTrain GoBus Transit Toronto Transportation","A New Class of Attack In Crypto Is Actively Exploited. TechTrees TTC","It appears that the TTC is becoming an attack Center for people as a woman was wounded during a fight in the bathroom of a TTC subway Station. Riders should be safe and not have to look over their shoulder at this point whats Torontos officials response and plan to address this"," question for the TTC: why do some TTC bus drivers stop for the geese crossing the road? do they care about the geese or are they just scared the geese will attack the bus","MakerDAO cofounder recommends DAIUSD depegging to limit attack surface BinanceTTC TTC TechTrees","More than 4.7M stolen in Uniswap fake token phishing attack BinanceTTC TTC TechTrees","Another crypto bridge attack: Nomad loses 190 million in chaotic hack BinanceTTC TTC TechTrees","Elderly man has ear chewed off in Portland train station attack  Toronto Sun   Toronto Transit TTC GoTransit GoTrain GoBus Transportation","When you ride Torontos public transportation TTC, you stay close to walls or pillars in case youre pushed onto the tracks while your eyes quickly and constantly look for places that may provide protection in case theres a knife, machete, gun or other weapon attack.","Tonight at 6: A final farewell to OPP Cst. Greg Pierzchala, killed while attending a call last week. Another violent subway attack amp; how the city plans to boost the TTCs budget. Plus, should Ontario follow BCs lead and implement a 3day cooling off period for homebuyers?","Just witnessed another attack on the TTC the third in my last 5 rides and its clear that the  has gotten more dangerous in 2022 and beyond   Its time for our municipal leaders to step up and deal with this asap and put boots on the ground","   Cause right now ttc employees refuse to attend assaults watched a man known for sexually assaulting children attack customers last night emergency alarm hit nobody showed up alarm cleared as people were being attacked.  ","Prophet of Doom pleads guilty in Brooklyn subway attack  CityNews Toronto   Toronto Transit TTC GoTransit GoTrain GoBus Transportation","bronze  How did she attack Ashley? Ashley shared with Candiace that she was abstaining from alcohol because she was TTC trying to conceive and a few minutes later she was throwing coronas back. ANYONE would be confused by that behavior when your actions dont match your words”,” JUST IN: TTC employee injured in knife attack,  say a transit worker had been stabbed near Keele Street and Gulliver Avenue, police tweeted Wednesday afternoon. NorthYork 1800222TIPS More here:    ","For the second time in a week, a TTC employee has been injured in a knife attack, according to police   ","TTC employee injured in knife attack, Toronto police say   For the second time in a week, a TTC employee has been injured in a knife attack, according to police. TOpoli TorontoTransit onpoli onlab canlab","TorontoPolice seek woman after knife attack on teen at TTC subway station  via     "," Cops seek woman after knife attack on teen at TTC subwaystation  ","Woman sought in knife attack on teen at TTC subwaystation  ","Woman sought in knife attack on teen at TTC subway station  ","TTC operator suffers minor injuries in knife attack  CityNews..another example of lawlessness in Toronto  ","TTC operator suffers minor injuries in knife attack  ","CityNews TTC operator suffers minor injuries in knife attack. Read Blog:  ","TTC operator hurt in attack with Exacto knife at Eglinton bus bay early this morning; minor injuries 88Toronto","TTC operator suffers minor injuries in knife attack    ","Knife attack and defence clinic tonight at TTC with Cst. John Irving. Thank you  ","October 5th: F.O. Guest instructor John Irving at TTC for first hour on  knife attack\/defence\/shock knife. Open to all Police Judo members.","UPDATE: Accused in TTC subway knife attack faces more charges  ","UPDATED: Bail hearing postponed for accused in TTC subway knife attack.   Toronto","Big lead in knifeattack on 2 sisters near Lawrence amp; Avenue yest. AM: Police released TTC pics of suspect in the vicious double sex assault","Man charged in TTC knife attack  ","Man charged in TTC knife attack: A young Toronto man was charged after a Sunday afternoon subway station knife att...  ","Man charged in TTC knife attack”, “TPS: reports of fight with weapon at TTC station, arrest made  CTV ...  CTV News Toronto   Toronto Transit TTC GoTransit GoTrain GoBus Transportation","Person arrested after reported fight with a weapon at TTC station, police say  ","why does the interviewer joke about pepper spray, which is a prohibited weapon in Canada.  ","Inflation is a war, and only way to win this war is by being TechTrees. TTC is the weapon that will protect you from inflation and keep you and your family safe. Win this battle with TTC. BinanceTTC  ","ayato I got Alhaitham and his weapon ","Stabbing, shootings, ttc assaults and this is what the mayor is doing. useless worstmayorever","Due to a random stabbing. Another reason not to take the ttc"," Interesting how there was a stabbing at one of your stations 16 hours ago when this account was activeand you said nothing about it.  Its almost like you are told not to talk about these things to improve ridership.   Me and my family will not ever be taking the ttc","Woman found at Dundas subway station with stabbing injuries  Global News   Toronto Transit TTC GoTransit GoTrain GoBus Transportation","Cool. Another stabbing on the TTC and another attempted push onto the tracks. Thanks John Tory. topoli"," 24  Read the article the stabbing did not occur on the TTC or at Dundas Station. The stabbing took place at York University near Osgoode Law School the victim then took the subway to Dundas Station.","Victim of stabbing incident located at Dundas subway station  CityNews Toronto   Toronto Transit TTC GoTransit GoTrain GoBus Transportation"," The incident happened at York U, 25 kms away. The media is trying to create fear riding the TTC harley  Stabbing, senior murdered, subway shoving in a day   LIBERALS WANT CONTROL OVER YOUR PERSONAL LIVES. HOW?  FEAR Can you separate the signal from the noise.","I was glad to get out and laugh tonight...Im also glad I got home safe ...I am quite frankly on edge after everything that has happened in my area and then hearing of the stabbing, the issues on TTC ...please be vigilant everyone ","The TTC can fvck all the way off a SA, another pushing incident on platform and just now rumour of a stabbing? Ffs","Stabbing at Dundas station. Fuck.  ttc delay stabbing ota","Toronto police investigate whether girls alleged subway assaults related to fatal stabbing  Windsor Star:   TTC GoTransit GoTrain GoBus Transit Toronto Transportation","Toronto police investigate whether girls alleged subway assaults related to fatal stabbing  Windsor Star   Toronto Transit TTC GoTransit GoTrain GoBus Transportation","KenLees friends say stabbingvictim was hero in protectinggirlfriend    Toronto TOpoli Crime security Police cops assault Investigation Murder Manslaughter homicide stabbing Teengirls Teenagers Teens girls TTC Publictransit Transit", "Police are looking for victims of alleged assaults by teen girls on the TTC in December. They have not confirmed if the incidents are linked to the death of a man who died after an alleged swarmingstyle stabbing by a group of teenage girls.  ","All this sounds so great... until the next stabbing our outage at the TTC.  The only question I have for  is... what happened to the SmartTrack?","Id love to see social workers tackle a man with a knife stabbing people on the TTC","TTC bus driver seriously hurt in stabbing following fare dispute toronto canada bhivenectar bhivelabs nectarsocial  "," Disgrace,no one is safe,home invasionsamp; cars stolen at gun point,TTC amp; subway stabbing,POLICE are gunned down.BILLIONS funded to Communist countries,millions of unvetted,unskilled immigrants are flooding cities,health care has crumbled,Highest taxes amp; inflation.TRUDEAU amp; J SINGH","We love you: Memorial for Toronto subway stabbing victim grows  CTV News Toronto:   TTC GoTransit GoTrain GoBus Transit Toronto Transportation","We love you: Memorial for Toronto subway stabbing victim grows  CTV News Toronto   Toronto Transit TTC GoTransit GoTrain GoBus Transportation”, “We love you: Memorial for Toronto subway stabbing victim grows  CP24:   TTC GoTransit GoTrain GoBus Transit Toronto Transportation","We love you: Memorial for Toronto subway stabbing victim grows  CP24   Toronto Transit TTC GoTransit GoTrain GoBus Transportation","Which TTC assault? In the stabbing incident at High Park station last week the police arrested a 52 year old man."," I decided to avoid as much TTC and use my car. Not save anymore after an every day headline of stabbing, pushing, dumping acid. I rather paid the overpriced parking.", "Its not safe.  avoid Canada. Lots of stabbing deaths here. TTC is not safe, condos are not safe and streets are not safe. Cancel FIFA and any international sports in Canada."]
co = cohere.Client('HiviLqEmyrSGCMosPNvtAoPCjTjau03Es3ktO3Cx')
response = co.classify(
  model='large',
  inputs=input_data, # why doesn't it generate classification for all entries in tweetdata? only generates one entry
  examples=[Example("Definitely \"shocking\". From reports that I saw after this; a female who was found stabbed last night @ Dundas Squ. it happened at a Univ. campus (stabbed in the abdomen) and another incident, too. Female saying \"they wished they saw TTC staff to talk to\", not necessarily police.", "avoid"), Example("Sad to hear about the woman pushed to the ground at Yonge and King by a random stranger and she has died. Good they have the person in custody.", "avoid"), Example("Thankfully, the man was not pushed onto the TTC tracks at Bloor earlier today. The man that tried to push him was arrested at Rosedale. Good on the TTC staff who recognized him and called it in. Way too much random violence happening.", "avoid"), Example("Victim of stabbing incident located at Dundas subway station", "avoid"), Example("Here we go again another incident today on TTC property at rosedale subway ! But again our mayor says its safe / I wonder if he would say this if one of his family members got hurt in these incidents ! Car Pool its safer if u can", "avoid"), Example("There was another attack on the TTC this morning. This one was hate motivated towards a specific community. I must have missed the incident by five mins or so. Be safe out there.", "avoid"), Example("Man pushed to tracks in argument at Bloor-Yonge Station in latest incident of TTC violence", "avoid"), Example("Also 2 people stabbed (1 dead) on the TTC, and another few slashed in the face by a knife in a random incident. My city is not safe.", "avoid"), Example("Which TTC assault? In the stabbing incident at High Park station last week the police arrested a 52 year old man.", "avoid"), Example("Police say a man who was reportedly stabbing seats onboard a Toronto subway train Friday afternoon hasn’t been located. No injuries have been reported. The incident comes one day after a fatal stabbing onboard a TTC subway train.", "avoid"), Example("ARREST MADE: Demonstration on Sat, Feb 5 Bedford Rd + Bloor St West Man, 34 arrested after throwing feces at another person. He was charged with: 1. Assault with a Weapon", "avoid"), Example("STABBING: Bloor St W + Roxton Rd at 12:22 am - At/near a bar - Reports of man stabbed multiple times - Unknown how serious - Police o/s - Have located victim - Will be taken to hospital - Officers searching area - 1 man now in custody - Weapon recovered", "avoid"), Example("Witnessed a bunch of young teenager girls attack each other with knives near bloor and ossington", "avoid"), Example("A suspect allegedly attacked a man wearing a turban at Yonge-Bloor subway station this morning", "avoid"), Example("Man arrested after 4 injured in hammer attack at Bloor subway station", "avoid"), Example("2 brothers at Yonge & Dundas Sq. in Toronto, Canada, who were giving da\'wah were violently assaulted by 3 men at 3:15 pm", "avoid"), Example("ASSAULT/STABBING - Hurontario St / Dundas St W Adult male stabbed being taken to trauma centre - Suspect w/ knife fled on foot - M/B, in his 30\'s, large build, all black clothing - Large police presence - Awaiting further information", "avoid"), Example("501 Queen: Minor delays eastbound near Queen St East at Jarvis St while we respond to a medical emergency.", "delay"), Example("Line 1 Yonge-University: Minor delays northbound at Rosedale due to a security incident.", "delay"), Example("101 Downsview Park: Minor delays near Sheppard Ave West at Chesswood Dr due to a blocked road.", "delay"), Example("47 Lansdowne: Detour southbound via College St, Dufferin St and Queen St W while we respond to a security incident.", "delay"), Example("501 Queen: No service between Long Branch Loop and Queen St East at Neville Park Blvd due to a security incident. 501 Queen service turning back at Kingston Road Loop.", "delay"), Example("what’s up with northbound subway? Very slow", "delay"), Example("hi TTC, your yonge university subway line southbound is very much delayed and slow. It is making continual frequent stops. This is really bad service. I am continously delayed using your transit system. Very disappointed.", "delay"), Example("Hi Phil, sorry to hear about your experience this evening. Delays regarding this route were posted on the web.", "delay"), Example("The delay we were experiencing on Line 2 at Islington Station is now clear. The delay was cleared by a TTC Staff Member.", "delay"), Example("Line 3 Scarborough: Minor delays between Kennedy and McCowan while we fix a mechanical problem.", "delay"), Example("Hi, sorry about the wait. The subway is running 3 minutes late and should be there in approx. 2 minutes.", "delay"), Example("Line 3 Scarborough: Minor delays at McCowan while we fix a mechanical problem.", "delay"), Example("501 Queen: Minor delays near Queen St East at Kingston Rd due to construction.", "delay"), Example("501 Queen Regular service has resumed between Dufferin St at Springhurst Ave and Queen St West at Dufferin St.", "delay"), Example("I was glad to get out and laugh tonight...I\'m also glad I got home safe ...I am quite frankly on edge after everything that has happened in my area and then hearing of the stabbing, the issues on TTC ...please be vigilant everyone ", "opinion"), Example("god I hate the TTC", "opinion"), Example("Being unreliable, unhelpful, unsafe, not adjusting to any road emergencies will do it for you. Every single person working for TTC management and the planning department needs to be a regular user of the service.", "opinion"), Example("maybe one day toronto will get its shit together and increase police presence to deter assault", "opinion"), Example("ttc sucks its slow everyday and shit happens all the time", "opinion"), Example("never a day without delay on the ttc woohoo", "opinion"), Example("Honest what hell is going on  gta so many scams,   people  getting  assault  in ttc property,", "opinion"), Example("How the ttc pretends to protect people in their facilities? A camera records events AFTER they happened! I have been victim of verbal assault and left before escalated to more. What about the incident on King station? TTC is not safe!", "opinion"), Example("This should not be assault. Anyone trying to push someone onto a live train track should be charged with attempted murder. It’s seems to violence is continuing on the TTC in 2023. To many criminals, addicted and mentally disturbed people just hanging around on TTC property", "opinion"), Example("Toronto police have released security images of a suspect being sought after a sexual assault that took place in the bathroom at a TTC subway station", "opinion"), Example("It’s amazing to me that we’re 15 months into this and the TTC would still rather pay fare inspectors to harass passengers than hire people to hand out masks and ensure riders are wearing them on TTC vehicles.", "opinion"), Example("because of continued failed communication and policies, this government really going to allow annoying rude ass people to harass others for wearing masks. people are already going at each other’s necks daily on the ttc. what a gd mess", "opinion"), Example("So we gotta pay more for shitty transit but the cops (both inside and outside the TTC) get more money to do nothing but harass poor folks, while 10 more outreach workers are hired to try fix a system that is intentionally broken. Got it.", "opinion"), Example("So many women on TikTok have stories of harassment on the TTC and lord knows the TTC will do absolutely nothing but harass you for the $3.25 How many stories have come out about the TTC this summer? Many. How have they responded? By bringing back fare enforcement officers.", "opinion"), Example("ttc drivers/employees are quick to harass customers or kick them off over bus fare or whatever they assume to be an attitude. but when commuters are being harassed by someone on the bus they\'re quiet and it makes them complicit.", "opinion"), Example("TTC \"lacks\" the recourses to safeguard transit users from being burned alive or pushed off the track but has a host of funds to harass and arrest random fare-dodgers", "opinion"), Example("Half of Toronto subway riders say crowding makes them feel unsafe, poll says", "opinion")])
label_output = response.classifications
col = []
avoid_tweet = []
for i in range(len(input_data)):
  label = re.findall('"([^"]*)"', str(label_output[i]))
  if label == ['avoid']:
    avoid_tweet.append(input_data[i])
    col.append([input_data[i], label])

fields = ['Tweet', 'Label']


with open('Tweet_Labels.csv', 'w') as f:
      
    csv_writer = csv.writer(f)
    csv_writer.writerow(fields)
    csv_writer.writerows(col)





In [ ]:
co = cohere.Client('HiviLqEmyrSGCMosPNvtAoPCjTjau03Es3ktO3Cx')
summary = []
for i in range(len(avoid_tweet)):
  response = co.generate(
    model='xlarge',
    prompt="Given a tweet, please return a summary of the tweet. Do not start a new tweet. \n Tweet: Also 2 people stabbed (1 dead) on the TTC, and another few slashed in the face by a knife in a random incident. My city is not safe.\n Summary: 3 people injured by knife on TTC\n Tweet: 2 brothers at Yonge & Dundas Sq. in Toronto, Canada, who were giving da'wah were violently assaulted by 3 men at 3:15 pm\n Summary: 2 brothers assaulted at Dundas Square\n Tweet: Thankfully, the man was not pushed onto the TTC tracks at Bloor earlier today. The man that tried to push him was arrested at Rosedale. Good on the TTC staff who recognized him and called it in. Way too much random violence happening.\n Summary: Man arrested for attempting to push someone onto tracks at Bloor \n Tweet: Police say a man who was reportedly stabbing seats onboard a Toronto subway train Friday afternoon hasn’t been located. No injuries have been reported. The incident comes one day after a fatal stabbing onboard a TTC subway train. \n Summary: Man stabbing seats on subway not found \n Tweet: There was another attack on the TTC this morning. This one was hate motivated towards a specific community. I must have missed the incident by five mins or so. Be safe out there. \n Summary: Another attack on the TTC this morning \n Tweet: Sad to hear about the woman pushed to the ground at Yonge and King by a random stranger and she has died. Good they have the person in custody.\n Summary: Woman pushed to ground at Yonge and King by random stranger and dies\n Tweet: Man charged in TTC knife attack: A young Toronto man was charged after a Sunday afternoon subway station knife attack\n Summary: Man charged in TTC knife attack\n Tweet: " + avoid_tweet[i] + " Summary: ", 
    max_tokens=15,
    temperature=0.3,
    k=0,
    p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop_sequences=["--"],
    return_likelihoods='NONE')
  items = str(response.generations[0].text)
  if 'Tweet' in items:
    idx = items.index('Tweet')
    final = items[:idx]
  else:
    final = items
  summary.append([final])
field = ['Alert']
with open('Alerts.csv', 'w') as f:
      
    csv_writer = csv.writer(f)
    csv_writer.writerow(field)
    csv_writer.writerows(summary)

